# Describe a PIV evaluation using openPIV

[OpenPIV](https://github.com/OpenPIV/openpiv-python) is an open-source PIV software. Let's use it to process the PIV-Challenge recording. Of course, you should have installed for this tutorial 

## Get image filenames from JSON-LD

In [1]:
import pivmetalib
import ontolutils
from pivmetalib import pivmeta

**Download files**

In [2]:
image_dists = ontolutils.query(pivmeta.PivImageDistribution, source='piv_challenge.jsonld',)
image_dist = image_dists[0]

In [3]:
zip_filename = image_dist.download()
zip_filename

WindowsPath('C.zip')

**unzip**

In [4]:
import zipfile
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('imgs')

**get filenames based on filename pattern**

In [5]:
import pathlib

In [6]:
image_dist.filenamePattern

'C[0-9][0-9][0-9]_[1,2].tif'

In [7]:
filenames = sorted(pathlib.Path('imgs').glob(image_dist.filenamePattern)) #image_dist.filenamePattern))
filenames

[WindowsPath('imgs/C001_1.tif'), WindowsPath('imgs/C001_2.tif')]

In [8]:
mask_dist = pivmeta.PivMaskDistribution.from_jsonld(source='piv_challenge.jsonld', limit=1)
mask_dist

PivMaskDistribution(id=_:N96eaa647e7744de18ccabdb2f48adaef, title=Mask file, download_URL=https://www.pivchallenge.org/pub/C/C.zip, media_type=https://www.iana.org/assignments/media-types/image/tiff, filenamePattern=Cmask_1.tif, compressedFormat=application/zip)

In [9]:
mask_filename = sorted(pathlib.Path('imgs').glob(mask_dist.filenamePattern)) #image_dist.filenamePattern))
mask_filename

[WindowsPath('imgs/Cmask_1.tif')]

## Perform evaluation with OpenPIV

In [ ]:
# !pip install openpiv

In [11]:
import openpiv
from openpiv import tools, pyprocess, validation, filters, scaling

# openpiv.__version__

ModuleNotFoundError: No module named 'openpiv'

We'll be re-using code from [openpiv's tutorial 1](https://nbviewer.org/github/OpenPIV/openpiv-python-examples/blob/main/notebooks/tutorial1.ipynb):

In [ ]:
# !pip install opencv-python

In [ ]:
import cv2
import numpy as np

frame_a = cv2.imread(str(filenames[0]), -1)
frame_b = cv2.imread(str(filenames[1]), -1)

# mask images
image_mask = cv2.imread(str(mask_filename), -1)
frame_a_masked = np.where(image_mask, frame_a, 0)
frame_b_masked = np.where(image_mask, frame_b, 0)

In [ ]:
import matplotlib.pyplot as plt

fig, _axs = plt.subplots(1,2,figsize=(12,10))
axs = _axs.ravel()
axs[0].imshow(frame_a,cmap=plt.cm.gray)
axs[1].imshow(frame_b,cmap=plt.cm.gray)
# axs[2].imshow(frame_a_masked, cmap=plt.cm.gray)
# axs[3].imshow(frame_b_masked, cmap=plt.cm.gray)

In [ ]:
winsize = 32 # pixels, interrogation window size in frame A
searchsize = 38  # pixels, search in image B
overlap = 12 # pixels, 50% overlap
dt = 1 # sec, arbitrary here as we have no information

u0, v0, sig2noise = pyprocess.extended_search_area_piv(frame_a.astype(np.int32), 
                                                       frame_b.astype(np.int32), 
                                                       window_size=winsize, 
                                                       overlap=overlap, 
                                                       dt=dt, 
                                                       search_area_size=searchsize, 
                                                       sig2noise_method='peak2peak')



In [ ]:
x, y = pyprocess.get_coordinates(image_size=frame_a.shape, 
                                 search_area_size=searchsize, 
                                 overlap=overlap )

In [ ]:
flags = validation.sig2noise_val( sig2noise, 
                                 threshold = 1.05 )

In [ ]:
# filter out outliers that are very different from the
# neighbours

u2, v2 = filters.replace_outliers( u0, v0, 
                                   flags,
                                   method='localmean', 
                                   max_iter=3, 
                                   kernel_size=3)

In [ ]:
# convert x,y to mm
# convert u,v to mm/sec

x, y, u3, v3 = scaling.uniform(x, y, u2, v2, 
                               scaling_factor = 1 ) # 1 microns/pixel (arbitrary, no info)

# 0,0 shall be bottom left, positive rotation rate is counterclockwise
x, y, u3, v3 = tools.transform_coordinates(x, y, u3, v3)

In [ ]:
#save in the simple ASCII table format
tools.save('exp1_001.txt', x, y, u3, v3, flags=flags)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(frame_a,cmap=plt.cm.gray, alpha=0.5)

v_abs = np.sqrt(u3**2+v3**2)

plt.rcParams['image.cmap'] = 'viridis'

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

q = ax.quiver(x, y, u3, v3, v_abs)
ax.quiver(x[flags], y[flags], u3[flags], v3[flags], color='r', label='invalid')
plt.colorbar(q, cax=cax)

## Describe the result file

First of all we need to describe the software that was used:

In [ ]:
from pivmetalib import prov

from pprint import pprint

In [ ]:
software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='OpenPIV',
        url='https://github.com/OpenPIV/openpiv-python',
    ),
    description='OpenPIV is an open source Particle Image Velocimetry analysis software written in Python and Cython',
    softwareVersion='0.25.0',#openpiv.__version__,
    has_documentation='https://openpiv.readthedocs.io/en/latest/',
)
pprint(software.model_dump(exclude_none=True))

In [ ]:
import pathlib
result_distribution = pivmeta.PivResultDistribution(
    downloadURL=pathlib.Path('exp1_001.txt')
)

In [ ]:
from ontolutils import M4I

In [ ]:
M4I.Researcher

In [ ]:
from pivmetalib import dcat, m4i
import rdflib

result_dataset = dcat.Dataset(
    distribution=result_distribution,
    hadRole=prov.Person(firstName='Matthias',
                        role=M4I.Researcher)
)
print(result_dataset.model_dump_jsonld(
    context={"@import": "https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld"}
)
)